In [1]:
import pandas as pd
import numpy as np

def normalize_feature(d):
    return d.apply(lambda col: (col - col.mean()) / col.std())

df0 = pd.read_csv('../data/house-price/train.csv')
df = normalize_feature(df0[['LotArea', 'BedroomAbvGr', 'SalePrice']])

ones = pd.DataFrame({'ones': np.ones(len(df))})
df = pd.concat([ones, df], axis=1)
df.head()

,ones,LotArea,BedroomAbvGr,SalePrice
0,1.0,-0.207071,0.163723,0.347154
1,1.0,-0.091855,0.163723,0.007286
2,1.0,0.073455,0.163723,0.535970
3,1.0,-0.096864,0.163723,-0.515105
4,1.0,0.375020,1.389547,0.869545


# 数据处理 获取 x y

In [2]:
import tensorflow as tf


In [3]:
X_data = np.array(df[df.columns[0:3]])
y_data = np.array(df[df.columns[-1]]).reshape(len(df), 1)

print(X_data.shape, type(X_data))
print(y_data.shape, type(y_data))

(1460, 3) <class 'numpy.ndarray'>
(1460, 1) <class 'numpy.ndarray'>


# 创建线性回归模型

In [4]:
alpha = 0.01 # 学习率
epoch = 100 # 训练全量数据集的轮数

with tf.name_scope("input"):
    # 输入 x, 形状[1460, 3]
    X = tf.placeholder(tf.float32, X_data.shape)
    # 输入 y, 形状[1460, 1]
    y = tf.placeholder(tf.float32, y_data.shape)

with tf.name_scope("hypothesis"):
    # 权重变量, 形状 [3, 1]
    W = tf.get_variable('weights', (X_data.shape[1], 1), initializer=tf.constant_initializer())
    
    # 假设函数 h(x) = w0*x0 + w1*x1 + w2*x2, 其中 x0=1
    # 推理值 y_pred
    y_pred = tf.matmul(X, W)

with tf.name_scope("loss"):
    # 损失函数-最小二乘法
    # tf.matmul(a, b, transpose_a=True) 表示: 矩阵a的转置乘以矩阵b, 
    # 损失函数 loss
    loss_op = 1 / (2 * len(X_data)) * tf.matmul((y_pred - y), (y_pred - y), transpose_a=True)

with tf.name_scope("train"):
    # 随机优化器 opt
    opt = tf.train.GradientDescentOptimizer(learning_rate=alpha)
    # 单轮训练操作
    train_op = opt.minimize(loss_op)


Instructions for updating:
Colocations handled automatically by placer.


In [5]:
with tf.Session() as sess:
    # 初始化全局变量
    sess.run(tf.global_variables_initializer())
    
    # 创建 FileWriter 实例
    writer = tf.summary.FileWriter("../summary/linear-reg-1/", sess.graph)
    # 开始训练模型
    for e in range(1, epoch+1):
        sess.run(train_op, feed_dict={X: X_data, y: y_data})
        if e % 10 == 0:
            loss, w = sess.run([loss_op, W], feed_dict={X: X_data, y: y_data})
            log_str = "Epoch %d \t Loss=%.4g \t Model: y = %.4gx1 + %.4gx2 + %.4g"
            print(log_str % (e, loss, w[1], w[2], w[0]))
writer.close()


Epoch 10 	 Loss=0.4908 	 Model: y = 0.02513x1 + 0.01594x2 + -1.906e-10
Epoch 20 	 Loss=0.4838 	 Model: y = 0.04768x1 + 0.03008x2 + -1.81e-10
Epoch 30 	 Loss=0.4781 	 Model: y = 0.06792x1 + 0.04263x2 + -2.293e-10
Epoch 40 	 Loss=0.4736 	 Model: y = 0.0861x1 + 0.05375x2 + -8.644e-11
Epoch 50 	 Loss=0.47 	 Model: y = 0.1024x1 + 0.06362x2 + -1.953e-10
Epoch 60 	 Loss=0.4671 	 Model: y = 0.1171x1 + 0.07236x2 + -6.403e-11
Epoch 70 	 Loss=0.4648 	 Model: y = 0.1302x1 + 0.08011x2 + -5.792e-11
Epoch 80 	 Loss=0.4629 	 Model: y = 0.142x1 + 0.08698x2 + -3.754e-11
Epoch 90 	 Loss=0.4614 	 Model: y = 0.1526x1 + 0.09306x2 + 1.965e-10
Epoch 100 	 Loss=0.4602 	 Model: y = 0.1622x1 + 0.09845x2 + 3.85e-10
